In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
import os
import json
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait

# Sessão de variaveis
path_to_downloads = "C:\\Users\\MrRobot\\Downloads\\music_download\\"
path_to_musics = "C:\\Users\\MrRobot\\Music\\"
service = Service(executable_path="../chromedriver.exe")
options = webdriver.ChromeOptions()
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36' # se tiver "headless" no user_agent o site bloqueia o acesso
options.add_argument('user-agent={0}'.format(user_agent))
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--mute-audio')
# options.add_argument("user-data-dir=C:\\Users\\MrRobot\\AppData\\Local\\Google\\Chrome\\User Data\\Profile 4") #Path to your chrome profile
prefs = {"download.default_directory" : path_to_downloads}
options.add_experimental_option("prefs",prefs)
c = webdriver.Chrome(service=service, options=options)
wdw = WebDriverWait(c, 40, 2)

def pegando_urls_no_ytb(link):
    """crawl da playlist do youtube e retorna ela em DICT"""
    try:
        c.get(link)

        # sleep(20)
        videos = c.find_elements(By.XPATH, "//a[@id = 'video-title']")
    
        urls = []
        for hrefs in videos:
            dict = {}
            dict['title'] = hrefs.get_attribute("innerHTML").replace('\n          ', '').replace('\n        ', '')
            dict['url'] = hrefs.get_attribute("href")
            urls.append(dict)

        print(f'total de URLs: {len(urls)}')
        return urls
    except:
        return "ERRO ao pegar URLS da playlist no Youtube."

def automacao_only_mp3(url_msc):
        """Recebe uma url de um video e Faz as ações dentro do site de download"""
        site = "https://pt.onlymp3.to/38/"
        c.get(site)

        search_bar = c.find_element(By.ID, "txtUrl")
        search_bar.send_keys(url_msc)
        
        sleep(1)
        convert_btn = c.find_element(By.CLASS_NAME, "start-btn")
        convert_btn.click()

        wdw.until(lambda c: c.find_element(By.XPATH, "//button[@class='btn']"), "Esperando btn de download")

        sleep(1)
        download_btn = c.find_element(By.XPATH, "//button[@class='btn']")
        download_btn.click()
        print('espera acabou')
        sleep(2)

def espera_download():
    """ Pergunta: Tem algum item em processo de download ou o diretório está vázio ? """
    sleep(3)
    mb = os.listdir(path_to_downloads)
          
    for item in mb:
        if item == 'desktop.ini':
            mb.remove('desktop.ini')
        if ".crdownload" in item:
            # Se tiver item em download 
            # print(item)
            return True

    if len(mb) == 0: #se diretório está vazio
        return True
        
    print(mb)
    return False

def renomear_arquivos():
    """Renomeia os arquivos no diretório de downloads"""
    mb = os.listdir(path_to_downloads)

    for i in mb:
        if i == 'desktop.ini':
            mb.remove('desktop.ini')
        new_i = i.replace('onlymp3.to - ', '')
        os.replace(path_to_downloads+i, f'{path_to_downloads}{new_i[:-35]}.mp3')

def mover_arquivos():
    """mover os arquivos do diretório de downloads para music"""

    mb = os.listdir(path_to_downloads)

    for i in mb:
        if i == 'desktop.ini':
            mb.remove('desktop.ini')
        os.replace(path_to_downloads+i, f'{path_to_musics}{i}')

def download_404():
    """
        Pergunta: ao clicar em download foi redirecionado para uma página de 404 ?
    """
    try:
        pagina_404 = c.find_element(By.XPATH, '//body[@id = "t"]')
        if bool(pagina_404):
            return True
    except:
        return False

In [10]:
# PROCESSO DE EXECUÇÃO:
link_playlist_ytb = 'https://www.youtube.com/playlist?list=PLB5S3eu0mZof9705EtpG8SeFACN2itGRP'

if 'urls_ytb' in globals(): # se a variavel já foi declarada...
    if len(urls_ytb) > 0:
        print(f'Não precisará fazer scrapy no ytb... Qt de Urls: {len(urls_ytb)}')
    else: # senão tem itens na lista...
        urls_ytb = pegando_urls_no_ytb(link_playlist_ytb)
else: # senão foi declarada a variavel
    urls_ytb = pegando_urls_no_ytb(link_playlist_ytb)

contador = 0
tentativas_download = 1
bad_request_404 = []
tomaram_timeout = []
while bool(urls_ytb) and tentativas_download < 5: # vai tentar 4x enquanto ter item na lista
    print(f'iniciando a {tentativas_download}° Tentativa')
    tentativas_download += 1
    for msc in urls_ytb:
        try:
            automacao_only_mp3(msc['url'])
            # Enquanto espera download... 
            timeout = 0
            while espera_download():
                timeout += 1
                sucesso_no_download = True
                print('Esperando o download terminar...') #ajuda a debugar :D
                if download_404():
                    # Quando der 404 ele pula pra próx musica
                    print(f'Problema de servidor na música {msc["title"]}')
                    if msc not in bad_request_404:
                        bad_request_404.append(msc)
                    sucesso_no_download = False
                    break
                if timeout > 30:
                    # Vai dar timeout se nao fizer o download em X segundos
                    sucesso_no_download = False
                    tomaram_timeout.append(msc)
                    print(f'TIMEOUT na música {msc["title"]}')
                    break
                sleep(2) # vezes a quantidade timeout = 60 segundos

            if sucesso_no_download:
                urls_ytb.remove(msc)
                renomear_arquivos()
                mover_arquivos()
                print(f"Download concluído: {msc['title']}")
                            
            contador += 1
            # Reiniciar browser quando quantidade de downloads for um divisor de 10
            dez_msc = contador % 5
            if dez_msc == 0 and contador != 0:
                print('reiniciando browser...')
                c.quit()
                c = webdriver.Chrome(service=service, options=options)
                wdw = WebDriverWait(c, 20, 2)

            print(f'Tentativa: {contador}. Número de itens para ser baixados: {len(urls_ytb)}')
        except:
            print(f'Erro na música: {msc["title"]}')
    

Não precisará fazer scrapy no ytb... Qt de Urls: 1
iniciando a 1° Tentativa
espera acabou
Esperando o download terminar...
Esperando o download terminar...
['onlymp3.to - Jorge & Mateus - 5 Regras (Clipe Oficial) [É Simples Assim]-VFWKrXxJEdM-192k-1654562942741.mp3']
Download concluído: Jorge &amp; Mateus - 5 Regras (Clipe Oficial) [É Simples Assim]
Tentativa: 1. Número de itens para ser baixados: 0


In [11]:
bool(urls_ytb)

False

In [12]:
print('urls q não foram baixadas:')
urls_ytb

urls q não foram baixadas:


[]

In [13]:
print('Falha na requisição: ')
bad_request_404

Falha na requisição: 


[]

In [14]:
print('timeout:')
tomaram_timeout

timeout:


[]

In [15]:
if bool(urls_ytb):
    print('há itens para ser baixado')
else:
    c.quit()